# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:
from sklearn.model_selection import train_test_split

# Create features and target data
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

- The random_state in the splitting function is a parameter used to control the randomness of the train-test split. Setting random_state=42 (or any other fixed number) makes sure that the split is the same every time you run the code. This reproducibility is useful for debugging, comparing models, or sharing results, as it ensures that you and others will work with the exact same training and testing subsets.

If you don't set the random_state, the split will be randomized differently each time you run the code, which can lead to variations in results, especially on smaller datasets.

- Setting a fixed random_state for the train-test split is useful primarily for reproducibility and consistency in machine learning experiments and comparisons. In general, setting random_state ensures that your experiments and findings are stable, transparent, and can be independently verified.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Define lists of numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define transformations for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define transformations for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Combine transformations into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ]
)

# Create a full pipeline with preprocessor and a model
pipe_simple = Pipeline([
    ('preprocess', preprocessor)
])
pipe_simple

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])]))])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [8]:
# Assuming 'preprocessor' is defined as in your previous code
# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Assign the Column Transformer for preprocessing
    ('classifier', RandomForestClassifier())  # Assign RandomForestClassifier
])

model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [11]:
X = adult_dt.drop(columns='income')  # Features
Y = adult_dt['income']  # Target variable

# Define lists of numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define transformations for numerical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

# Define transformations for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Combine transformations into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ]
)

# Define the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Assign the Column Transformer for preprocessing
    ('classifier', RandomForestClassifier())  # Assign RandomForestClassifier
])

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define performance metrics for evaluation
scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

# Evaluate the model pipeline using cross-validation
res_pipeline_dict = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Convert results to a DataFrame and assign an experiment number
res_pipeline = pd.DataFrame(res_pipeline_dict).assign(experiment=1)

# Calculate mean results for training and validation
train_results = {
    'train_neg_log_loss': res_pipeline['train_neg_log_loss'].mean(),
    'train_roc_auc': res_pipeline['train_roc_auc'].mean(),
    'train_accuracy': res_pipeline['train_accuracy'].mean(),
    'train_balanced_accuracy': res_pipeline['train_balanced_accuracy'].mean(),
}

val_results = {
    'val_neg_log_loss': res_pipeline['test_neg_log_loss'].mean(),
    'val_roc_auc': res_pipeline['test_roc_auc'].mean(),
    'val_accuracy': res_pipeline['test_accuracy'].mean(),
    'val_balanced_accuracy': res_pipeline['test_balanced_accuracy'].mean(),
}

# Print training results
print("Training Results:")
for metric, value in train_results.items():
    print(f"{metric}: {value:.4f}")

# Print validation results
print("\nValidation Results:")
for metric, value in val_results.items():
    print(f"{metric}: {value:.4f}")

# Display the cross-validation results DataFrame
res_pipeline

c:\Users\khali\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\khali\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Results:
train_neg_log_loss: -0.0818
train_roc_auc: 1.0000
train_accuracy: 1.0000
train_balanced_accuracy: 1.0000

Validation Results:
val_neg_log_loss: -0.3662
val_roc_auc: 0.9031
val_accuracy: 0.8527
val_balanced_accuracy: 0.7724


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
0,8.353692,0.157539,-0.367245,-0.080705,0.894641,1.0,0.844914,1.000000,0.757625,1.000000,1
1,8.248856,0.171818,-0.343856,-0.082006,0.905705,1.0,0.853743,0.999904,0.781413,0.999801,1
2,8.123931,0.150788,-0.378848,-0.081429,0.902532,1.0,0.849520,1.000000,0.764471,1.000000,1
3,8.150491,0.167496,-0.375659,-0.082437,0.906208,1.0,0.855443,1.000000,0.775461,1.000000,1
4,8.266140,0.154627,-0.365396,-0.082201,0.906363,1.0,0.859666,1.000000,0.782871,1.000000,1


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [12]:
# Create a fold-level results DataFrame with selected columns
fold_results = res_pipeline[['train_neg_log_loss', 'train_roc_auc', 'train_accuracy', 'train_balanced_accuracy',
                              'test_neg_log_loss', 'test_roc_auc', 'test_accuracy', 'test_balanced_accuracy']]

# Sort by negative log loss of the test (validation) set
fold_results_sorted = fold_results.sort_values(by='test_neg_log_loss')

# Display the sorted fold-level results
fold_results_sorted.reset_index(drop=True, inplace=True)  # Reset index for better display
print(fold_results_sorted)

   train_neg_log_loss  train_roc_auc  train_accuracy  train_balanced_accuracy  \
0           -0.081429            1.0        1.000000                 1.000000   
1           -0.082437            1.0        1.000000                 1.000000   
2           -0.080705            1.0        1.000000                 1.000000   
3           -0.082201            1.0        1.000000                 1.000000   
4           -0.082006            1.0        0.999904                 0.999801   

   test_neg_log_loss  test_roc_auc  test_accuracy  test_balanced_accuracy  
0          -0.378848      0.902532       0.849520                0.764471  
1          -0.375659      0.906208       0.855443                0.775461  
2          -0.367245      0.894641       0.844914                0.757625  
3          -0.365396      0.906363       0.859666                0.782871  
4          -0.343856      0.905705       0.853743                0.781413  


Calculate the mean of each metric. 

In [14]:
# Calculate the mean of each metric
mean_results = fold_results.mean()

# Display the sorted fold-level results
fold_results_sorted.reset_index(drop=True, inplace=True)  # Reset index for better display
print("Sorted Fold-Level Results:")
print(fold_results_sorted)

# Display mean results
print("\nMean Results:")
print(mean_results[['train_neg_log_loss', 'train_roc_auc', 'train_accuracy', 'train_balanced_accuracy',
                    'test_neg_log_loss', 'test_roc_auc', 'test_accuracy', 'test_balanced_accuracy']])

Sorted Fold-Level Results:
   train_neg_log_loss  train_roc_auc  train_accuracy  train_balanced_accuracy  \
0           -0.081429            1.0        1.000000                 1.000000   
1           -0.082437            1.0        1.000000                 1.000000   
2           -0.080705            1.0        1.000000                 1.000000   
3           -0.082201            1.0        1.000000                 1.000000   
4           -0.082006            1.0        0.999904                 0.999801   

   test_neg_log_loss  test_roc_auc  test_accuracy  test_balanced_accuracy  
0          -0.378848      0.902532       0.849520                0.764471  
1          -0.375659      0.906208       0.855443                0.775461  
2          -0.367245      0.894641       0.844914                0.757625  
3          -0.365396      0.906363       0.859666                0.782871  
4          -0.343856      0.905705       0.853743                0.781413  

Mean Results:
train_neg_log_l

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [16]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the model pipeline on the training data
model_pipeline.fit(X_train, Y_train)

# Obtain prediction probabilities for the test set
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate performance metrics
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])  # Only the positive class probabilities
neg_log_loss_value = log_loss(Y_test, Y_pred_proba)
accuracy = accuracy_score(Y_test, model_pipeline.predict(X_test))
balanced_accuracy = balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))

# Store the results in a dictionary
test_results = {
    'negative_log_loss': neg_log_loss_value,
    'roc_auc': roc_auc,
    'accuracy': accuracy,
    'balanced_accuracy': balanced_accuracy
}

# Display the results
print("Test Performance Metrics:")
print(test_results)

Test Performance Metrics:
{'negative_log_loss': 0.36220906647994133, 'roc_auc': 0.9072150375289063, 'accuracy': 0.8595117457392907, 'balanced_accuracy': 0.7847674655539587}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.